In [1]:
import datafaucet as dfc

In [2]:
# start the engine
project = dfc.project.load()

created SparkEngine
Init engine "spark"
Configuring packages:
  -  mysql:mysql-connector-java:8.0.12
  -  org.apache.hadoop:hadoop-aws:3.2.1
Configuring conf:
  -  spark.hadoop.fs.s3a.access.key : ****** (redacted)
  -  spark.hadoop.fs.s3a.endpoint : http://minio:9000
  -  spark.hadoop.fs.s3a.impl : org.apache.hadoop.fs.s3a.S3AFileSystem
  -  spark.hadoop.fs.s3a.path.style.access : true
  -  spark.hadoop.fs.s3a.secret.key : ****** (redacted)
Connecting to spark master: local[1]
Engine context spark:2.4.4 successfully started


In [3]:
dfc.list('mysql', 'races').data.collect()

,name,type
0,id,bigint
1,name,string
2,trade,double
3,fight,double
4,greedy,double


In [16]:
df = dfc.load('races', 'mysql')

In [17]:
df.data.grid()

,id,name,trade,fight,greedy
0,0,human,0.3,0.7,0.5
1,1,dwarf,0.6,0.4,0.8
2,2,orc,0.2,0.8,0.5
3,3,elf,0.8,0.2,0.2


In [18]:
df = (df
    .cols.find(by_type='double').rand()
    .cols.get('name').hashstr(salt='foobar')
    .rows.
)

In [19]:
df.limit(3).data.grid()

,id,name,trade,fight,greedy
0,0,D420C715,0.462660,0.687999,0.653009
1,1,FD200E54,0.044966,0.318078,0.642276
2,2,1ECC2478,0.124710,0.106253,0.624643


In [15]:
df.printSchema()

root
 |-- id: long (nullable = true)
 |-- name: string (nullable = true)
 |-- trade: double (nullable = false)
 |-- fight: double (nullable = false)
 |-- greedy: double (nullable = false)



In [16]:
df.save('races', 'minio')

True

In [17]:
dfc.list('minio', 'races').data.grid()

,name,type
0,_SUCCESS,FILE
1,part-00000-352df1b1-4682-44fd-b288-c8349233af4...,FILE


INFO:kafka.client:Closing idle connection 1001, last active 26487699 ms ago
INFO:kafka.client:Closing idle connection 1001, last active 5074769 ms ago
